**BERTurk TEXT CLASSIFICATION - Immigration**


Alkan Can Mollaibrahimoglu CSS MA Koç University



**Experiment 2:** tweets without preprocessing


**Relevant count:** 345

**Irrelevant count:** 1035

### Importing Necessary Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

### Loading Relevant and Irrelevant Data Sets

In [ ]:
df=pd.read_excel('/content/final_data_labels.xlsx')

In [ ]:
df=df.drop_duplicates(subset=['_id'])

In [ ]:
df=df[['Tweet','label']]
df

,Tweet,label
0,zurnanın zırt dediği yerde tamda burası bugüne...,1
1,kusura bakmada bundan yıl önce çocukken bende ...,1
2,ülkemizin zor günlerinde tapu kadastro teknike...,0
3,türk misafirperverliğini aramaktayım misafire ...,0
4,irkçılık cezalandırılmağı suya karışmış zehir ...,1
...,...,...
15312,mülteciler sadece türkiye’nin değil bütün düny...,1
15313,iğdir vali̇si̇nden i̇bretli̇k uygulama olay çı...,1
15314,mülteci sorunu merkezi hükümeti ilgilendiren u...,1
15315,yaralanan çocukların fotoğraflarını gördüm kor...,1


In [ ]:
df= df.dropna(subset=['Tweet', 'label'])


In [ ]:
df['label'].value_counts()

,count
label,
0,14970
1,345


### Sampling Irrelevant Set

In [ ]:
df_relevant=df[df['label']==1]

In [ ]:
df_relevant

,Tweet,label
0,zurnanın zırt dediği yerde tamda burası bugüne...,1
1,kusura bakmada bundan yıl önce çocukken bende ...,1
4,irkçılık cezalandırılmağı suya karışmış zehir ...,1
6,çocuk fabrikası kurdunuz dabizim haberimiz mi ...,1
7,ırkçısınız sığınmacı düşmanısınız faşistsiniz ...,1
...,...,...
15312,mülteciler sadece türkiye’nin değil bütün düny...,1
15313,iğdir vali̇si̇nden i̇bretli̇k uygulama olay çı...,1
15314,mülteci sorunu merkezi hükümeti ilgilendiren u...,1
15315,yaralanan çocukların fotoğraflarını gördüm kor...,1


In [ ]:
df_irrelevant=df[df['label']==0]

In [ ]:
df_irrelevant=df_irrelevant.sample(n=1035,random_state=5)





### Train-Test-Validation Split






In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Combine the relevant and irrelevant dataframes
df_combined = pd.concat([df_relevant, df_irrelevant])

# Split the data into training (60%) and temporary data (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    df_combined['Tweet'],  # Drop the label column for X
    df_combined['label'],  # y is just the label column
    test_size=0.4,  # 40% for the temp set
    random_state=40,  # Ensuring reproducibility
    stratify=df_combined['label']  # Stratify split based on the labels
    )

# Split the temporary data into validation and test sets (50% each of temp data)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,  # 50% of temp data to each of validation and test sets
    random_state=40,  # Ensuring reproducibility
    stratify=y_temp  # Stratify split based on the labels
)


### Import Necessary Libraries

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
import random
import numpy as np
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
set_seed(42)  #

### Verification

In [ ]:
print(f"Training Set: {X_train.shape[0]} tweets")
print(f"Testing Set: {X_test.shape[0]} tweets")
print(f"Validation Set: {X_val.shape[0]} tweets")

Training Set: 828 tweets
Testing Set: 276 tweets
Validation Set: 276 tweets


In [ ]:
y_train.value_counts()

label
0    621
1    207
Name: count, dtype: int64

In [ ]:
y_test.value_counts()

label
0    207
1     69
Name: count, dtype: int64

In [ ]:
y_val.value_counts()

label
0    207
1     69
Name: count, dtype: int64

### Berttürk Prediction

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch

# Define the compute_metrics function for F1 score
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {"f1": f1_score(p.label_ids, predictions, average='binary')}  # You can change average to 'binary' if it's binary classification

# Load the tokenizer and model specifically for Turkish tweets
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-cased")
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-128k-cased", num_labels=2)  # Adjust num_labels as per your task

# Define a dataset class
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Function to tokenize data
def tokenize_data(X, tokenizer):
    return tokenizer(X.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')  # Adjusted max_length for tweets

# Tokenize the data
train_encodings = tokenize_data(X_train, tokenizer)
val_encodings = tokenize_data(X_val, tokenizer)
test_encodings = tokenize_data(X_test, tokenizer)

# Create datasets
train_dataset = TweetDataset(train_encodings, y_train.tolist())
val_dataset = TweetDataset(val_encodings, y_val.tolist())
test_dataset = TweetDataset(test_encodings, y_test.tolist())

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',            # Output directory
    evaluation_strategy="epoch",       # Evaluation is done at the end of each epoch
    save_strategy="epoch",             # Save model at the end of each epoch
    learning_rate=2e-5,                # Defines the learning rate
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=16,     # Batch size for evaluation
    num_train_epochs=3,                # Number of training epochs
    weight_decay=0.01,                 # Weight decay for regularization
    load_best_model_at_end=True,       # Load the best model when finished training
    metric_for_best_model='f1'         # Use F1 score to evaluate the best model
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Use F1 score for evaluation
)

# Train the model
trainer.train()

# Evaluate the model on the test set
test_results = trainer.evaluate(test_dataset)
print(test_results)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/740M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,F1
1,No log,0.280153,0.762712
2,No log,0.174839,0.910448


<ipython-input-17-8e0518411fca>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-8e0518411fca>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,F1
1,No log,0.280153,0.762712
2,No log,0.174839,0.910448
3,No log,0.198632,0.899225


<ipython-input-17-8e0518411fca>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-8e0518411fca>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.12350074201822281, 'eval_f1': 0.9343065693430658, 'eval_runtime': 98.4727, 'eval_samples_per_second': 2.803, 'eval_steps_per_second': 0.183, 'epoch': 3.0}


### Evaluation

In [ ]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)
print(classification_report(y_test, predicted_labels, target_names=['Irrelevant', 'Relevant']))

<ipython-input-17-8e0518411fca>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

  Irrelevant       0.98      0.98      0.98       207
    Relevant       0.94      0.93      0.93        69

    accuracy                           0.97       276
   macro avg       0.96      0.95      0.96       276
weighted avg       0.97      0.97      0.97       276



### Model Save

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_save_path = '/content/drive/My Drive/saved_models/BERTurk_immigration_2'

# Save the model and tokenizer
model.save_pretrained(model_save_path,safe_serialization=False)
tokenizer.save_pretrained(model_save_path)

Mounted at /content/drive


('/content/drive/My Drive/saved_models/BERTurk_immigration_2/tokenizer_config.json',
 '/content/drive/My Drive/saved_models/BERTurk_immigration_2/special_tokens_map.json',
 '/content/drive/My Drive/saved_models/BERTurk_immigration_2/vocab.txt',
 '/content/drive/My Drive/saved_models/BERTurk_immigration_2/added_tokens.json',
 '/content/drive/My Drive/saved_models/BERTurk_immigration_2/tokenizer.json')